In [1]:

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

/opt/miniconda3/envs/lauz24/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/miniconda3/envs/lauz24/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/miniconda3/envs/lauz24/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
from huggingface_hub import hf_hub_download

/opt/miniconda3/envs/lauz24/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
repo_id = "lmstudio-community/Llama-3.2-3B-Instruct-GGUF"
filename = "Llama-3.2-3B-Instruct-Q8_0.gguf"

# repo_id = "mav23/Llama_3.2_1B_Intruct_Tool_Calling_V2-GGUF"
# filename = "llama_3.2_1b_intruct_tool_calling_v2.Q8_0.gguf"


downloaded_file = hf_hub_download(repo_id=repo_id, filename=filename)

In [36]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=None,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=downloaded_file,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 31 key-value pairs and 255 tensors from /Users/wesleymonteith/.cache/huggingface/hub/models--lmstudio-community--Llama-3.2-3B-Instruct-GGUF/snapshots/f72420da9a2c36818f0b00f35adf0bab65e9b9c5/Llama-3.2-3B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 3B
ll

In [37]:
response = llm.complete("Hello! Can you tell me a poem about cats and dogs?")
print(response.text)


llama_print_timings:        load time =    5024.98 ms
llama_print_timings:      sample time =      11.58 ms /   183 runs   (    0.06 ms per token, 15800.38 tokens per second)
llama_print_timings: prompt eval time =    5024.76 ms /    70 tokens (   71.78 ms per token,    13.93 tokens per second)
llama_print_timings:        eval time =    8328.17 ms /   182 runs   (   45.76 ms per token,    21.85 tokens per second)
llama_print_timings:       total time =   13647.89 ms /   252 tokens


 

Here is a poem about cats and dogs:

Cats and dogs, a perfect pair,
Living together, without a care.
They play and cuddle, and snuggle up tight,
A beautiful friendship, a wondrous sight.

The cat's soft purrs, a soothing sound,
The dog's wagging tail, a joyful round.
They chase and romp, and have so much fun,
A happy duo, beneath the bright sun.

But when the day is done, and it's time to rest,
The cat and dog, snuggle up in their nest.
They dream of chases, and playtime too,
A peaceful slumber, with a heart that's true.

So here's to the cats, and the dogs so dear,
A perfect pair, a friendship so clear.
May their bond grow strong, and their love shine bright,
A beautiful friendship, a wondrous sight.


In [29]:
response_iter = llm.stream_complete("How many cars are there?")
for response in response_iter:
    print(response.delta, end="", flush=True)

 
There are 5 cars.

Llama.generate: 54 prefix-match hit, remaining 9 prompt tokens to eval

llama_print_timings:        load time =    2358.22 ms
llama_print_timings:      sample time =       0.53 ms /     8 runs   (    0.07 ms per token, 15180.27 tokens per second)
llama_print_timings: prompt eval time =      70.02 ms /     9 tokens (    7.78 ms per token,   128.54 tokens per second)
llama_print_timings:        eval time =     122.33 ms /     7 runs   (   17.48 ms per token,    57.22 tokens per second)
llama_print_timings:       total time =     199.39 ms /    16 tokens


In [34]:
llm.metadata.is_function_calling_model

False

---
# Function Calling

In [18]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event


class InputEvent(Event):
    input: list[ChatMessage]


class ToolCallEvent(Event):
    tool_calls: list[ToolSelection]


class FunctionOutputEvent(Event):
    output: ToolOutput

In [19]:
from typing import Any, List

from llama_index.core.llms.function_calling import FunctionCallingLLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step


class FuncationCallingAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: FunctionCallingLLM | None = None,
        tools: List[BaseTool] | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []

        self.llm = llm or OpenAI()
        assert self.llm.metadata.is_function_calling_model

        self.memory = ChatMemoryBuffer.from_defaults(llm=llm)
        self.sources = []

    @step
    async def prepare_chat_history(self, ev: StartEvent) -> InputEvent:
        # clear sources
        self.sources = []

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        self.memory.put(user_msg)

        # get chat history
        chat_history = self.memory.get()
        return InputEvent(input=chat_history)

    @step
    async def handle_llm_input(
        self, ev: InputEvent
    ) -> ToolCallEvent | StopEvent:
        chat_history = ev.input

        response = await self.llm.achat_with_tools(
            self.tools, chat_history=chat_history
        )
        self.memory.put(response.message)

        tool_calls = self.llm.get_tool_calls_from_response(
            response, error_on_no_tool_call=False
        )

        if not tool_calls:
            return StopEvent(
                result={"response": response, "sources": [*self.sources]}
            )
        else:
            return ToolCallEvent(tool_calls=tool_calls)

    @step
    async def handle_tool_calls(self, ev: ToolCallEvent) -> InputEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.get_name(): tool for tool in self.tools}

        tool_msgs = []

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            additional_kwargs = {
                "tool_call_id": tool_call.tool_id,
                "name": tool.metadata.get_name(),
            }
            if not tool:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Tool {tool_call.tool_name} does not exist",
                        additional_kwargs=additional_kwargs,
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                self.sources.append(tool_output)
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=tool_output.content,
                        additional_kwargs=additional_kwargs,
                    )
                )
            except Exception as e:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Encountered error in tool call: {e}",
                        additional_kwargs=additional_kwargs,
                    )
                )

        for msg in tool_msgs:
            self.memory.put(msg)

        chat_history = self.memory.get()
        return InputEvent(input=chat_history)

In [21]:
llm.metadata.is_function_calling_model

False

In [20]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI


def add(x: int, y: int) -> int:
    """Useful function to add two numbers."""
    return x + y


def multiply(x: int, y: int) -> int:
    """Useful function to multiply two numbers."""
    return x * y


tools = [
    FunctionTool.from_defaults(add),
    FunctionTool.from_defaults(multiply),
]

agent = FuncationCallingAgent(
    llm=llm, tools=tools, timeout=120, verbose=True
)

ret = await agent.run(input="Hello!")

AssertionError: 